<b><h1><center>IE406: Machine Learning<center>Group: 14</center><h1></b>
<h3><center>Tipsi Jadav: 201801091</center></h3>
<h3><center>Tanishqa Lalwani: 201801139</center></h3>
<h3><center>Astha Shah: 201801169</center></h3>
<h3><center>Shreya Joshi: 201801174</center></h3>
<h3><center>Maulina Raina: 201801186</center></h3>

In [1]:
# ! pip install --upgrade albumentations

In [2]:
'''
from google.colab import drive
drive.mount('/content/drive')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

#**Importing necessary libraries**

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import cv2
import albumentations as A

In [5]:
import xml.etree.ElementTree as ET

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
import warnings
warnings.filterwarnings('ignore')

#**Importing Datasets from Kaggle**

##First Datasat

In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d andrewmvd/car-plate-detection
! unzip \*.zip

#Second Dataset

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d aritrag/license
! unzip \*.zip

#First Dataset 

In [10]:
def resize_image(img_arr, bboxes, h, w):
  transform = A.Compose([A.Resize(height=h, width=w, always_apply=True)], bbox_params=A.BboxParams(format='pascal_voc'))
 
  transformed = transform(image=img_arr, bboxes=bboxes)

  return transformed

In [11]:
read_image = []
xmin1 = []
ymin1 = []
xmax1 = []
ymax1 = []

In [12]:
list_of_images = os.listdir('images')
list_of_images = [x.replace('.png', '') for x in list_of_images]

In [13]:
for file in list_of_images:
  image_read1 = cv2.imread(f"images/{file}.png", 1)

  read_image.append(image_read1)

  root = ET.parse(f"annotations/{file}.xml").getroot()

  xmn=[]
  ymn=[]
  xmx=[]
  ymx=[]
  for type_tag in root.findall('object'):   
    xmn.append(int(type_tag.find('bndbox/xmin').text))
    ymn.append(int(type_tag.find('bndbox/ymin').text))
    xmx.append(int(type_tag.find('bndbox/xmax').text))
    ymx.append(int(type_tag.find('bndbox/ymax').text))
 
  xmin1.append(xmn)
  ymin1.append(ymn)
  xmax1.append(xmx)
  ymax1.append(ymx)

In [14]:
images1 = []
cordinates1 = []

for i in range(0,len(read_image)):
  bboxes_og=[]
  for j in range(0,len(xmin1[i])):
    bboxes_og.append(np.array([round(xmin1[i][j]), ymin1[i][j], xmax1[i][j], ymax1[i][j], 0]))
  
  bboxes_og = np.array(bboxes_og)
  bboxes_og = np.reshape(bboxes_og, newshape=(len(xmin1[i]), 5))
  
  temp = resize_image(read_image[i], bboxes_og, 512, 512)
  images1.append(temp["image"])
  cordinates1.append(temp["bboxes"])

#Second Dataset 

In [15]:
data = pd.read_csv('license.csv')

In [ ]:
images2 = []
cordinates2 = []

for index, row in data.iterrows():
  
  path = row['image_name']
  path = path.replace('../input/license/', '')
  if path == 'Cars/car107.jpg':
    continue 
  
  print(index)
  image_read2 = cv2.imread(path, 1)
  shape_im = image_read2.shape

  temp_xmin = int(row['top_x']*shape_im[1])
  temp_ymin = int(row['top_y']*shape_im[0])
  temp_xmax = int(row['bottom_x']*shape_im[1])
  temp_ymax = int(row['bottom_y']*shape_im[0])  

  bboxes_og = np.array([temp_xmin, temp_ymin, temp_xmax, temp_ymax, 0])
  bboxes_og = np.reshape(bboxes_og, newshape=(1, 5))

  temp = resize_image(image_read2, bboxes_og, 512, 512)
  images2.append(temp["image"])
  cordinates2.append(temp["bboxes"])

#**Combining dataset**

In [17]:
images = []
cordinates = []

len_tot = len(images1) + len(images2)

j = 0
for i in range(len_tot):
  if i >= len(images1):
    j = i - len(images1)
    images.append(images2[j])
    cordinates.append(cordinates2[j])
  else:
    j = i
    images.append(images1[j])
    cordinates.append(cordinates1[j])

In [18]:
for i in range(len(cordinates)):
  for j in range(len(cordinates[i])):
    cordinates[i][j] = [round(num) for num in cordinates[i][j]]

#**Data visualisation**

In [19]:
color = (255, 255, 255)
thickness = 2

In [ ]:
for i in range(100):
  image_new = images[i]
  cordinates_new = cordinates[i]

  for i in range(len(cordinates_new)):
    xmin = cordinates_new[i][0]
    ymin = cordinates_new[i][1] 
    xmax = cordinates_new[i][2]
    ymax = cordinates_new[i][3]

    start = (xmin, ymin)
    end = (xmax, ymax)
    
    image_new = cv2.rectangle(image_new, start, end, color, thickness)

  
  plt.figure(figsize=(10,10))
    
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(image_new)

plt.show()

#**Train/Test split**

In [ ]:
images_train, images_test, cordinates_train, cordinates_test = train_test_split(images, cordinates, test_size=0.2, random_state=42)

Train

In [ ]:
# master_train = "/content/drive/MyDrive/IE406-Machine_Learning/Project/Codes/YOLO/TensorFlow-2.x-YOLOv3/custom_data/Train"

In [ ]:
# url_train = []

# for i in range(len(images_train)):  
#   path_train = f"{master_train}/Cars{i}.jpg"
#   cv2.imwrite(path_train, images_train[i])
  
#   for j in range(len(cordinates_train[i])):
#     temp = cordinates_train[i][j]
#     path_train = f"{path_train} {temp[0]},{temp[1]},{temp[2]},{temp[3]},{0}" 

#   url_train.append(path_train)

In [ ]:
# url_file = open("/content/drive/MyDrive/IE406-Machine_Learning/Project/Codes/YOLO/TensorFlow-2.x-YOLOv3/model_data/coco/license_plate_train.txt", "w")
# for element in url_train:
#   url_file.write(element + "\n")  
# url_file.close()

Test


In [ ]:
# master_test = "/content/drive/MyDrive/IE406-Machine_Learning/Project/Codes/YOLO/TensorFlow-2.x-YOLOv3/custom_data/Test"

In [ ]:
# url_test = []

# for i in range(len(images_test)): 
#   path_test = f"{master_test}/Cars{i}.jpg"
#   cv2.imwrite(path_test, images_test[i])
  
#   for j in range(len(cordinates_test[i])):
#     temp = cordinates_test[i][j]
#     path_test = f"{path_test} {temp[0]},{temp[1]},{temp[2]},{temp[3]},{0}" 

#   url_test.append(path_test)

In [ ]:
# url_file = open("/content/drive/MyDrive/IE406-Machine_Learning/Project/Codes/YOLO/TensorFlow-2.x-YOLOv3/model_data/coco/license_plate_test.txt", "w")
# for element in url_test:
#   url_file.write(element + "\n")  
# url_file.close()